In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
data = pd.read_csv('titanic.csv')

In [2]:
data.head()

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def checkAdult(age):
    if age>=18:
        return "Adult"
    else:
        return "Child"
    
data["Adult/Child"]=data["Age"].apply(checkAdult)
data.head() 

,PassengerId,Survived,Pclass,Name,Gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Adult/Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult


In [4]:
data.groupby(['Gender'])['Survived'].sum()

Gender
female    233
male      109
Name: Survived, dtype: int64

In [5]:
trainingData=data[["Pclass","Adult/Child","Gender","Survived"]]
trainingData.head()

,Pclass,Adult/Child,Gender,Survived
0,3,Adult,male,0
1,1,Adult,female,1
2,3,Adult,female,1
3,1,Adult,female,1
4,3,Adult,male,0


In [6]:
len(trainingData)

891

In [7]:
trainingData = trainingData.dropna()
len(trainingData)

891

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(trainingData, test_size = 0.2)

In [9]:
train.head()

,Pclass,Adult/Child,Gender,Survived
170,1,Adult,male,0
125,3,Child,male,1
879,1,Adult,female,1
855,3,Adult,female,1
420,3,Child,male,0


In [10]:
test.head()

,Pclass,Adult/Child,Gender,Survived
53,2,Adult,female,1
369,1,Adult,female,1
231,3,Adult,male,0
106,3,Adult,female,1
539,1,Adult,female,1


In [11]:
len(train)

712

In [12]:
len(test)

179

In [13]:
def entropy(col):
    
    elements,freq = np.unique(col,return_counts = True)
    entropy = np.sum([(-freq[i]/np.sum(freq))*np.log2(freq[i]/np.sum(freq)) for i in range(len(elements))])
    return entropy

In [14]:
def InfoGain(data,split_attribute_name,target_name="Survived"):
    """
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    parent_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = parent_entropy - Weighted_Entropy
    return Information_Gain
       

In [15]:
def ID3(data, originaldata, features, target_attribute_name="Survived", parent_node_class=None):
    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#

    # If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    # If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name], return_counts=True)[1])]

    # If the feature space is empty, return the mode target feature value of the direct parent node --> Note that
    # the direct parent node is that node which has called the current run of the ID3 algorithm and hence
    # the mode target feature value is stored in the parent_node_class variable.
    elif len(features) == 0:
        return parent_node_class

     # If none of the above holds true, grow the tree!
    # Set the default value for this node --> The mode target feature value of the current node
    parent_node = np.unique(data[target_attribute_name])[np.argmax(
        np.unique(data[target_attribute_name], return_counts=True)[1])]
    # Select the feature which best splits the dataset
    item_value = []
    for feat in features:
        item_value.append(InfoGain(data, feat))
    # item_value = InfoGain(data, features)
    # #Return the information gain values for the features in the dataset
    best_feature_index = np.argmax(item_value)
    best_feature = features[best_feature_index]
    # print(best_feature)
    #     #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
    # #gain in the first run
    tree = {best_feature: {}}
    # #Remove the feature with the best inforamtion gain from the feature space
    # data[best_features] ...values remove best
    for x in np.unique(data[best_feature]):
        subdata = data[data[best_feature] != x]
        subdata = subdata.drop([best_feature], 1)
        features = [i for i in features if i != best_feature]
        # print(features)
        tree[best_feature][x] = ID3(
            subdata, originaldata, features, target_attribute_name, parent_node)
    
    return (tree)
    # #Grow a branch under the root node for each possible value of the root node feature
    # for values in data[best_features]:

    #     #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
    # subdata = data.bestvalue.drop values
    #     #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
    #     ID3(sub_data , )
    #     #Add the sub tree, grown from the sub_dataset to the tree under the root node
    # tree[best_features][value] = subtree

    #   return (tree)

In [16]:
tree = ID3(train,train,train.columns[:-1])
print(tree)

{'Gender': {'female': {'Pclass': {1: {'Adult/Child': {'Adult': 0, 'Child': 0}}, 2: {'Adult/Child': {'Adult': 0, 'Child': 0}}, 3: {'Adult/Child': {'Adult': 0, 'Child': 0}}}}, 'male': {'Pclass': {1: {'Adult/Child': {'Adult': 1, 'Child': 1}}, 2: {'Adult/Child': {'Adult': 1, 'Child': 1}}, 3: {'Adult/Child': {'Adult': 1, 'Child': 1}}}}}}


C:\Users\ahmad\AppData\Local\Temp\ipykernel_1364\3809026840.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  subdata = subdata.drop([best_feature], 1)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_1364\3809026840.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  subdata = subdata.drop([best_feature], 1)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_1364\3809026840.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  subdata = subdata.drop([best_feature], 1)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_1364\3809026840.py:38: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  subdata = subdata.drop([best_feature], 1)
C:\Users\ahmad\AppData\Local